In [0]:
%disableCompletion
import TensorFlow

# Modifying machine learning models


In [0]:
struct Model: Layer {
    var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6))
    var maxpool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
    var flatten = Flatten<Float>()
    var dense = Dense<Float>(inputSize: 36 * 6, outputSize: 10)

    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: conv, maxpool, flatten, dense)
    }
}


In [0]:
// Use random training data.
let x = Tensor<Float>(randomNormal: [10, 16, 16, 3])
let y = Tensor<Int32>(rangeFrom: 0, to: 10, stride: 1)

In [0]:
var model = Model()
let opt = SGD(for: model)
Context.local.learningPhase = .training

In [0]:
for i in 1...10 {
    print("Starting training step \(i)")
    let (loss, grads) = valueWithGradient(at: model) { model -> Tensor<Float> in
        let logits = model(x)
        return softmaxCrossEntropy(logits: logits, labels: y)
    }
    print("Loss: \(loss)")
    opt.update(&model, along: grads)
}

Now: modify the model to use skip connections!

# Writing custom layers


In [0]:
// A custom layer type that's similar to a Dense layer, but with an extra bias
// term.
struct DoubleBiasDense: Layer {
  // TODO(saeta): FILL ME IN!
}


In [0]:
struct Model2: Layer {
    var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6))
    var maxpool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
    var flatten = Flatten<Float>()
    var dense = DoubleBiasDense(inputSize: 36 * 6, outputSize: 10)

    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: conv, maxpool, flatten, dense)
    }
}


In [0]:
var model = Model2()
let opt = SGD(for: model)

for i in 1...10 {
    print("Starting training step \(i)")
    let (loss, grads) = valueWithGradient(at: model) { model -> Tensor<Float> in
        let logits = model(x)
        return softmaxCrossEntropy(logits: logits, label: y)
    }
    print("Loss: \(loss)")
    opt.update(&model, along: grads)
}

Voilà.... almost.

# Experimenting with batch sizes


In [0]:
for i in 1...10 {
    print("Starting training step \(i)")
    var grads = Model2.TangentVector.zero
    for j in 1...4 {
        print("Running substep \(j)")
        let stepGrad = gradient(at: model) { model -> Tensor<Float> in
            let logits = model(x)
            return softmaxCrossEntropy(logits: logits, labels: y)
        }
        grads += stepGrad
    }
    opt.update(&model, along: grads)
}